# Rigid Body Simulator

Credit to Professor Bo Zhu, on whose COSC 89 Course Notes the following code is based on

## Geometry in space

UNFINISHED

In [205]:
import taichi as ti
import numpy as np

ti.init()

# variables and fields to keep track of
num_particles = 1
dt = 1e-3
substeps = 10
mass = 1

# Inertial tensor
I = 10
# position - center of mass
x = ti.Vector.field(n=2, dtype=ti.f32, shape=num_particles)
# x = ti.Vector([0.5, 0.5])
# vertices - relative to center of mass -- will have to change if more than one solid
vertices_0 = ti.Vector.field(n=2, dtype=ti.f32, shape=num_particles*3)
rotated_vertices = ti.Vector.field(n=2, dtype=ti.f32, shape=num_particles*3)
# force
force = ti.Vector.field(n=3, dtype=ti.f32, shape=num_particles)
# torque
torque = ti.Vector.field(n=3, dtype=ti.f32, shape=num_particles)
# velocity
v = ti.Vector.field(n=3, dtype=ti.f32, shape=num_particles)
# orientation
theta = ti.field(dtype=ti.f32, shape=num_particles)
# angular velocity
w = ti.Vector.field(n=3, dtype=ti.f32, shape=num_particles)

@ti.kernel
def init_pos():
    x[0] = ti.Vector([0.5, 0.5])
    # equilateral triangle
    vertices_0[0] = ti.Vector([0, 0.1])
    vertices_0[1] = ti.Vector([-0.0866, -0.05])
    vertices_0[2] = ti.Vector([0.0866, -0.05])
    rotated_vertices[0] = ti.Vector([0, 0.1])
    rotated_vertices[1] = ti.Vector([-0.0866, -0.05])
    rotated_vertices[2] = ti.Vector([0.0866, -0.05])
    # initialize velocities
    v[0] = ti.Vector([0, 0, 0])
    # initialize force
    force[0] = ti.Vector([0,0,0])
    
@ti.func
def clear_force():
    for i in range(num_particles):
        force[i] = ti.Vector([0.0, 0.0, 0.0])
    
    
@ti.func
def calculate_force():
    clear_force()
    # gravity
    for i in range(num_particles):
        force[i] = force[i] - ti.Vector([0, -2, 0])

@ti.func
def rotate():
    pass

@ti.func
def calculate_torque():
    pass

@ti.kernel
def substep():
    calculate_force()
    calculate_torque()
    # update velocity
    v[0] = v[0] + dt * force[0] / mass
    # update angular velocity
    w[0] = w[0] + dt * torque[0] / I
    # update position
    x[0] = x[0] + dt * v[0]
    # update orientation
    theta[0] = theta[0] * dt * w[0]
    # update vertices
    rotate()

def main():
    gui = ti.GUI('Rigid Body', (600, 600))
    # initialize vertices
    init_pos()
    
    while gui.running:
        for e in gui.get_events(ti.GUI.PRESS):
            pass
            
        # Update for a time step
        for step in range(substeps):
            substep()

        gui.triangle(a=x[0]+rotated_vertices[0], b=x[0]+rotated_vertices[1], c=x[0]+rotated_vertices[2], color=0xED553B)
        
        gui.show()
    
if __name__ == '__main__':
    main()

[Taichi] Starting on arch=x64


TaichiCompilationError: 
On line 76 of file "/var/folders/y7/rxbvhmr9307f345mkrl28_xw0000gn/T/ipykernel_5370/1594515034.py", in substep:
    x[0] = x[0] + dt * v[0]
           ^^^^^^^^^^^^^^^^
Traceback (most recent call last):
  File "/Users/andresibarra/opt/anaconda3/lib/python3.9/site-packages/taichi/lang/ast/ast_transformer_utils.py", line 24, in __call__
    return method(ctx, node)
  File "/Users/andresibarra/opt/anaconda3/lib/python3.9/site-packages/taichi/lang/ast/ast_transformer.py", line 683, in build_BinOp
    node.ptr = op(node.left.ptr, node.right.ptr)
  File "/Users/andresibarra/opt/anaconda3/lib/python3.9/site-packages/taichi/lang/ast/ast_transformer.py", line 669, in <lambda>
    ast.Add: lambda l, r: l + r,
  File "/Users/andresibarra/opt/anaconda3/lib/python3.9/site-packages/taichi/lang/common_ops.py", line 35, in __add__
    return ops.add(self, other)
  File "/Users/andresibarra/opt/anaconda3/lib/python3.9/site-packages/taichi/lang/ops.py", line 55, in wrapped
    r

Code from stackoverflow on finding the inertial tensor using triangles

https://stackoverflow.com/questions/41592034/computing-tensor-of-inertia-in-2d